In [ ]:
import pandas as pd
import numpy as np

# Define the tanks with static properties
tanks = pd.DataFrame({
    'Tank': ['FeedTank1', 'FeedTank2', 'SlurryTank', 'FiltrateTank'],
    'Capacity_Liters': [6500, 6500, 6500, 6500],  # Capacities in liters
    'HH': [0.9, 0.9, 0.9, 0.9],  # Example trigger levels
    'H': [0.8, 0.8, 0.8, 0.8],
    'L': [0.2, 0.2, 0.2, 0.2],
    'LL': [0.1, 0.1, 0.1, 0.1],
})
#Pre-calculated values
tanks['ÚsedCapacity'] = tanks['Capacity_Liters']*(tanks['H']-tanks['L'])

# Initialize the dynamic properties (start all tanks at 20% capacity)
tanks['Level_Liters'] = tanks['Capacity_Liters'] * 0.2
tanks['Level_Percent'] = (tanks['Level_Liters'] / tanks['Capacity_Liters'])*100

# Display the initialized tanks DataFrame
print(tanks)

           Tank  Capacity_Liters   HH    H    L   LL  ÚsedCapacity  \
0     FeedTank1             6500  0.9  0.8  0.2  0.1        3900.0   
1     FeedTank2             6500  0.9  0.8  0.2  0.1        3900.0   
2    SlurryTank             6500  0.9  0.8  0.2  0.1        3900.0   
3  FiltrateTank             6500  0.9  0.8  0.2  0.1        3900.0   

   Level_Liters  Level_Percent  
0        1300.0           20.0  
1        1300.0           20.0  
2        1300.0           20.0  
3        1300.0           20.0  


In [ ]:
# Define durations in minutes for different events
event_durations = {
    'Feed_Duration_FeedTank1': 10,  # 10 minutes of feeding into FeedTank1
    'Feed_Duration_FeedTank2': 12,  # 12 minutes of feeding into FeedTank2
    'Slurry_Duration': 15,  # 15 minutes of slurrying
    'Filtrate_Duration': 8,  # 8 minutes of filtrating
    'Wash_Duration': 5  # 5 minutes of washing
}

print(event_durations)


{'Feed_Duration_FeedTank1': 10, 'Feed_Duration_FeedTank2': 12, 'Slurry_Duration': 15, 'Filtrate_Duration': 8, 'Wash_Duration': 5}


In [ ]:
# Define the rates in liters per minute
rates = {
    'FeedRate_EMU': 300,  #
    'OutputRate_EMU': 330,  # More by CO2
    'FeedRate_LAROX': 7000,  # Spec of pump
    'OutputRate_LAROX': 6585,  # Less due to solids
    'FlushRate_LAROX': 7000,  # 20 L/min for washing processes
    'WashRate_LAROX': 7000,  # 20 L/min for washing processes
    'ClothRate_LAROX': 7000  # 20 L/min for washing processes
}

print(rates)


{'FeedRate_EMU': 300, 'OutputRate_EMU': 330, 'FeedRate_LAROX': 7000, 'OutputRate_LAROX': 6585, 'FlushRate_LAROX': 7000, 'WashRate_LAROX': 7000, 'ClothRate_LAROX': 7000}


In [ ]:
# Define durations in minutes for different events
event_durations = {
    'Duration_Fill_Feedtank':  tanks.loc[tanks['Tank'] == 'FeedTank1', 'ÚsedCapacity'].values[0]
    /rates['FeedRate_EMU'],  #Feed till full

    'Duration_Fill_Larox': 2,  # later linked to density and cake size
      'Duration_Flush_Larox': 2,  # later linked to density and cake size
      'Duration_Wash_Larox': 2,  # later linked to density and cake size
      'Duration_Cloth_Larox': 2,  # later linked to density and cake size
      'Duration_Cake_Larox': 2,  # later linked to density and cake size
}

print(event_durations)


{'Duration_Fill_Feedtank': 13.000000000000002, 'Duration_Fill_Larox': 2, 'Duration_Flush_Larox': 2, 'Duration_Wash_Larox': 2, 'Duration_Cloth_Larox': 2, 'Duration_Cake_Larox': 2}


In [ ]:
# Define the event array with details
events = [
    {'Tank': 'FeedTank1', 'Rate': rates['FeedRate_EMU'], 'Duration': event_durations['Duration_Fill_Feedtank']},
    {'Tank': 'FeedTank2', 'Rate': rates['FeedRate_EMU'], 'Duration': event_durations['Duration_Fill_Feedtank']},
    {'Tank': 'SlurryTank', 'Rate': -rates['OutputRate_EMU'], 'Duration': event_durations['Duration_Fill_Larox']},
]

print(pd.DataFrame(events))


         Tank  Rate  Duration
0   FeedTank1   300      13.0
1   FeedTank2   300      13.0
2  SlurryTank  -330       2.0


In [ ]:
# Simulation over discrete time steps
time_steps = max(event['Duration'] for event in events)  # Determine the longest event duration
log = []

for t in range(time_steps):
    for event in events:
        if t < event['Duration']:
            # Update the tank level based on the event's rate
            tank = event['Tank']
            rate = event['Rate']
            tanks.loc[tanks['Tank'] == tank, 'Level_Liters'] += rate

            # Ensure levels are within 0 and Capacity
            tanks.loc[tanks['Level_Liters'] > tanks['Capacity_Liters'], 'Level_Liters'] = tanks['Capacity_Liters']
            tanks.loc[tanks['Level_Liters'] < 0, 'Level_Liters'] = 0

            # Update the percentage level
            tanks['Level_Percent'] = (tanks['Level_Liters'] / tanks['Capacity_Liters']) * 100

    # Log the state of the tanks at each time step
    log.append(tanks.copy())

# Convert the log to a DataFrame for analysis
log_df = pd.concat(log, keys=range(time_steps), names=['Time_Step'])
print(log_df)


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [ ]:
import heapq
import pandas as pd


# static during simulation can be dynamic on settings (solids rate, converstion etc)
# define static tank properties: name, capacity, HH/H/L/LL_perc
# define initial dynamic tank properies: level_ltrs, level_perc,
# define static rates of flows: make batch, feed emu, output emu, feed filtrate larox, etc
# define static duration of events: Larox timings

# que of events
# loop over events,
  # determine timestep for this step (shortest timestep of events)
  # update tank levels,
  # determine triggers for new events,
  # define new events by determine dydnamic rates/durations, add to que
  # define

#triggers
# fill feedbatch: if tank is <xx , not in use, and time is between 6-(22-fill_time)
      # feetanks should have a status_Flag
# start EMU: if feed_tank > xx, keep running until slurry > xx or feedtank < xx and other tank is < xx
# swith feedtank: if feed_tank <xx and other tank is > xx
# start Larox: if slurry is > level, Filtrate is < level, trigger 1 cycle. Later add: pause during cycle if slurry < xx or Filtrate is > xx
# start filtrate discharge: triggered by feedbatch?
# drain filtrate?


# Define the tanks with static properties
tanks = pd.DataFrame({
    'Tank': ['FeedTank1', 'FeedTank2', 'SlurryTank', 'FiltrateTank'],
    'MaxCapacity_Liters': [1000, 1000, 500, 800],
    'Start_EMU_Level': [200, 200, 100, 160],
    'Stop_EMU_Level': [800, 800, 400, 640],
    'Start_Filter_Level': [500, 500, 250, 400],
    'Stop_Filter_Level': [900, 900, 450, 720],
})

# Initialize the dynamic properties (start all tanks at 20% capacity)
tanks['Level_Liters'] = tanks['MaxCapacity_Liters'] * 0.2
tanks['Level_Percent'] = (tanks['Level_Liters'] / tanks['MaxCapacity_Liters']) * 100
tanks['UsedCapacity_Liters'] = tanks['MaxCapacity_Liters'] * tanks['H'] - tanks['Level_Liters']

# Define the rates in liters per minute
rates = {
    'FeedRate_FeedTank1': 50,
    'FeedRate_FeedTank2': 60,
    'SlurryRate': 40,
    'FiltrateRate': 30,
    'WashRate': 20
}

# Initialize the event queue
event_queue = []

# Function to schedule events
def schedule_event(event_time, event_type, tank_name, rate):
    heapq.heappush(event_queue, (event_time, event_type, tank_name, rate)) #event_time is basically duration

# Initial events for filling the feed tanks
for tank in ['FeedTank1', 'FeedTank2']:
    used_capacity = tanks.loc[tanks['Tank'] == tank, 'UsedCapacity_Liters'].values[0]
    feed_rate = rates[f'FeedRate_{tank}']
    fill_duration = used_capacity / feed_rate
    schedule_event(fill_duration, 'stop_filling', tank, feed_rate)

# Initialize first events
#fill tank 1
    used_capacity = tanks.loc[tanks['Tank'] == tank, 'UsedCapacity_Liters'].values[0]
    feed_rate = rates[f'FeedRate_{tank}']
    fill_duration = used_capacity / feed_rate
    schedule_event(fill_duration, 'stop_filling', tank, feed_rate)

fill_duration =
schedule_event(fill_duration, 'stop_filling', tank, feed_rate)

# Function to process events
def process_event(event):
    event_time, event_type, tank_name, rate = event
    print(f"Processing event: {event_type} for {tank_name} at time {event_time}")

    if event_type == 'stop_filling':
        # Update the tank level
        tanks.loc[tanks['Tank'] == tank_name, 'Level_Liters'] += rate * event_time
        tanks.loc[tanks['Tank'] == tank_name, 'Level_Percent'] = (
            tanks.loc[tanks['Tank'] == tank_name, 'Level_Liters'] / tanks.loc[tanks['Tank'] == tank_name, 'MaxCapacity_Liters']
        ) * 100
        tanks.loc[tanks['Level_Liters'] > tanks['MaxCapacity_Liters'], 'Level_Liters'] = tanks['MaxCapacity_Liters']

        # Potentially trigger a new event based on the new level
        new_level = tanks.loc[tanks['Tank'] == tank_name, 'Level_Liters'].values[0]
        if new_level >= tanks.loc[tanks['Tank'] == tank_name, 'Start_Filter_Level'].values[0]:
            # Schedule a filtering event, for example
            filtrate_rate = rates['FiltrateRate']
            filtrate_duration = new_level / filtrate_rate
            schedule_event(event_time + filtrate_duration, 'start_filtration', tank_name, -filtrate_rate)

# Simulation loop
log = []
current_time = 0

while event_queue:
    # Get the next event
    next_event = heapq.heappop(event_queue)
    event_time = next_event[0]

    # Advance time to the event
    current_time = event_time

    # Process the event
    process_event(next_event)

    # Log the state after processing the event
    log.append(tanks.copy())

# Convert the log to a DataFrame for analysis
log_df = pd.concat(log, keys=[f"Step {i}" for i in range(len(log))], names=['Simulation Step'])
print(log_df)


Processing event: stop_filling for FeedTank2 at time 13.333333333333334
Processing event: stop_filling for FeedTank1 at time 16.0
Processing event: start_filtration for FeedTank2 at time 46.66666666666667
Processing event: start_filtration for FeedTank1 at time 49.333333333333336
                           Tank  Capacity_Liters  Start_EMU_Level  \
Simulation Step                                                     
Step 0          0     FeedTank1             1000              200   
                1     FeedTank2             1000              200   
                2    SlurryTank              500              100   
                3  FiltrateTank              800              160   
Step 1          0     FeedTank1             1000              200   
                1     FeedTank2             1000              200   
                2    SlurryTank              500              100   
                3  FiltrateTank              800              160   
Step 2          0     FeedTan

In [ ]:
import heapq
import pandas as pd
import numpy as np


# static during simulation can be dynamic on settings (solids rate, converstion etc)
# define static tank properties: name, capacity, HH/H/L/LL_perc
# define initial dynamic tank properies: level_ltrs, level_perc,
# define static rates of flows: make batch, feed emu, output emu, feed filtrate larox, etc
# define static duration of events: Larox timings

# que of events
# function for defining events
# loop over events,
  # determine timestep for this step (shortest timestep of events)
  # update tank levels,
  # determine triggers for new events,
  # define new events by determine dydnamic rates/durations, add to que
  #

# Define the tanks with static properties
tanks = pd.DataFrame({
    'Tank': ['FeedTank1', 'FeedTank2', 'SlurryTank', 'FiltrateTank'],
    'MaxCapacity': [6500, 6500, 6500, 6500],  # Capacities in liters
    'HH': [0.9, 0.9, 0.9, 0.9],
    'H': [0.8, 0.8, 0.8, 0.8],
    'L': [0.2, 0.2, 0.2, 0.2],
    'LL': [0.1, 0.1, 0.1, 0.1],
})


# Define the rates in liters per hour
rates = {
    'Feed_EMU': 300,  #
    'Output_EMU': 330,  # More by CO2
    'Feed_LAROX': 7000,  # Spec of pump
    'Output_LAROX': 6585,  # Less due to solids, later depending on settings
    'Flush_LAROX': 7000,  #
    'Wash_LAROX': 7000,  #
    'Cloth_LAROX': 7000  #
}
for key in rates:
    rates[key] /= 60  # Convert to liters per minute


# Define durations in minutes for different events
durations = {
    'Fill_Larox': 2,  # later linked to density and cake size
    'Flush_Larox': 2,  # Static
    'Wash_Larox': 2,  # later linked wash settings
    'Cloth_Larox': 2,  # later linked to pressure settings
    'Cake_Larox': 2,  # Static
}

# Initialize the event queue
event_queue = []

# Initialize the dynamic properties (start all tanks at 20% capacity)
tanks['Level_Liters'] = tanks['MaxCapacity'] * 0.2
tanks['Level_Percent'] = (tanks['Level_Liters'] / tanks['MaxCapacity'])*100


# Function to schedule events
def schedule_event(event_duration, event_type, tank_name, rate):
    heapq.heappush(event_queue, (event_duration, event_type, tank_name, rate))

# Initial events for filling the feed tanks
for tank in ['FeedTank1']:
    available_capacity = tanks.loc[tanks['Tank'] == tank, 'UsedCapacity_Liters'].values[0]
    feed_rate = rates[f'FeedRate_{tank}']
    fill_duration = used_capacity / feed_rate
    print(fill_duration)
    schedule_event(fill_duration, 'stop_filling', tank, feed_rate)

# Function to process events
def process_event(event):
    event_duration, event_type, tank_name, rate = event
    print(f"Processing event: {event_type} for {tank_name} at time {event_duration}")

    if event_type == 'stop_filling':
        # Update the tank level
        tanks.loc[tanks['Tank'] == tank_name, 'Level_Liters'] += rate * event_duration
        tanks.loc[tanks['Tank'] == tank_name, 'Level_Percent'] = (
            tanks.loc[tanks['Tank'] == tank_name, 'Level_Liters'] / tanks.loc[tanks['Tank'] == tank_name, 'Capacity_Liters']
        ) * 100
        tanks.loc[tanks['Level_Liters'] > tanks['Capacity_Liters'], 'Level_Liters'] = tanks['Capacity_Liters']

        # Potentially trigger a new event based on the new level
        new_level = tanks.loc[tanks['Tank'] == tank_name, 'Level_Liters'].values[0]
        if new_level >= tanks.loc[tanks['Tank'] == tank_name, 'Start_Filter_Level'].values[0]:
            # Schedule a filtering event, for example
            filtrate_rate = rates['FiltrateRate']
            filtrate_duration = new_level / filtrate_rate
            schedule_event(event_duration + filtrate_duration, 'start_filtration', tank_name, -filtrate_rate)

# Simulation loop
log = []
current_time = 0

while event_queue:
    # Get the next event
    next_event = heapq.heappop(event_queue)
    event_duration = next_event[0]

    # Advance time to the event
    current_time = event_duration

    # Process the event
    process_event(next_event)

    # Log the state after processing the event
    log.append(tanks.copy())

# Convert the log to a DataFrame for analysis
log_df = pd.concat(log, keys=[f"Step {i}" for i in range(len(log))], names=['Simulation Step'])
print(log_df)


